# Adiabatic Flame Example

This example uses `ctwrap` for one-dimensional adiabatic premixed flame simulations defined by the simulation module `adiabatic_flame` (`ctwrap.modules.adiabatic_flame`).

## 1. Function/Class Definitions

In [ ]:
import cantera as ct
import ctwrap as cw
import json

## 2. Simulations

### 2.1 Create Simulation Object

Uses a module in the `modules` folder. Modules do not have to depend on `ctwrap`.

In [ ]:
sim = cw.Simulation.from_module(cw.modules.adiabatic_flame)
sim

### 2.2 Create Simulation Handler

In [ ]:
fname = '../../yaml/adiabatic_flame.yaml'
s = cw.SimulationHandler.from_yaml(fname, verbosity=1)
s

### 2.3 Run Simulation Tasks in Parallel

In [ ]:
%%time
s.run_parallel(sim, verbosity=0)

## 3. Create Graphical Output

In [ ]:
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
col = mpl.rcParams['axes.prop_cycle'].by_key()['color']
n_col = len(col)

### 3.1 Load Data

In [ ]:
data = h5py.File(s.output_name, 'r')

In [ ]:
data.keys()

In [ ]:
data.attrs.keys()

In [ ]:
var = json.loads(data.attrs['strategy'])
var

In [ ]:
default = json.loads(data.attrs['defaults'])
default

In [ ]:
up = default.get('upstream')
up

In [ ]:
phi = var['sequence'].get('upstream.phi')
phi

In [ ]:
mix = {k.split('<')[0]: dict(data[k]['flame']) for k in data if 'mix' in k}
mlt = {k.split('<')[0]: dict(data[k]['flame']) for k in data if 'multi' in k}

In [ ]:
mix.keys()

### 3.2 Plot

Plot flame speeds and temperature profiles

In [ ]:
# create figure
fig, ax = plt.subplots(1)

# plot results
u_mix = [ mix[t]['velocity'][0] for t in mix]
u_mlt = [ mlt[t]['velocity'][0] for t in mlt]

ax.plot(phi, u_mix, marker='o', 
            linestyle='none', label='mixture-averaged') 
ax.plot(phi, u_mlt, marker='s', 
            markerfacecolor='none',
            linestyle='none', label='multi-component') 

# add title/axis labels
ax.set_xlabel('Equivalence ratio (-)')
ax.set_ylabel('Flame speed (m/s)')

ax.set_title('Adiabatic premixed flame simulation for {} with {}'.format(up['fuel'],up['oxidizer']))

# add legend
leg = ax.legend(loc='lower right',#prop={'size':11},
                title='Transport Model') # % mech)
_ = leg.draw_frame(False) 

In [ ]:
phi

In [ ]:
# create figure and adjust size
fig, ax = plt.subplots(1)
fig.set_size_inches(6.,6.)

# plot results
for i, key in enumerate(mix.keys()):
    df = mix[key]
    label = key.split('_')[1]
    ax.plot(1000*df['grid'][:], df['T'][:], color=col[i % n_col], 
            linestyle='-', label=label) 
for i, key in enumerate(mlt.keys()):
    df = mlt[key]
    ax.plot(1000*df['grid'][:], df['T'][:], color=col[i % n_col], 
            linestyle=':', label=label) 

# add title/axis labels
ax.set_xlabel('Position (mm)')
ax.set_ylabel('Temperature (K)')

# add legend
leg = ax.legend(loc='lower right',#prop={'size':11},
                title='Equivalence ratio', ncol=2) # % mech)
_ = leg.draw_frame(False) 